# <font color="#49699E" size=40>Not so Fast: Modelling the Diffusion of Complex Contagions</font>
# LEARNING OBJECTIVES
# LEARNING MATERIALS
# INTRODUCTION


# FROM SIMPLE TO COMPLEX CONTAGIONS


# BEYOND LOCAL NEIGHBORHOODS: NETWORK EFFECTS AND THRESHOLDS


## Complex Contagions, Narrow Bridges, Wide Bridges and Thresholds


# THRESHOLD MODELS FOR COMPLEX CONTAGIONS

## Imports

In [ ]:
import pandas as pd
pd.set_option("display.notebook_repr_html", False)
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.utils import multi_runs

from dcss.plotting import custom_seaborn
custom_seaborn()

from dcss.networks import *

bluetooth_contact = pd.read_csv("../data/copenhagen_networks_study/cns_bluetooth_filtered.csv", sep=',')
g_bluetooth_contact = nx.from_pandas_edgelist(bluetooth_contact, 'user_a', 'user_b', create_using=nx.Graph())
g_bluetooth_contact.name = 'CNS Bluetooth Contact'
print(nx.info(g_bluetooth_contact))

In [ ]:
infect_sets = [rand_infection_set(g_bluetooth_contact, 0.1) for x in range(500)]
infect_sets_2 = add_to_infection_set(infect_sets, 0.05, g_bluetooth_contact)

In [ ]:
%%capture
thresh_model = ep.ThresholdModel(g_bluetooth_contact)
thresh_config = mc.Configuration()

threshold = 0.1
fraction_infected = .1

thresh_config.add_model_parameter("fraction_infected", fraction_infected)

for n in g_bluetooth_contact.nodes():
    thresh_config.add_node_configuration("threshold", n, threshold)
    
thresh_model.set_initial_status(thresh_config)

threshold_trends = multi_runs(thresh_model,
                              execution_number=500,
                              iteration_number=40,
                              infection_sets=infect_sets, 
                              nprocesses=4)

In [ ]:
visualize_trends(threshold_trends,
                 network=g_bluetooth_contact,
                 states=[0, 1],
                 labels=['Not Activated', 'Activated'],
                 proportion=True,
                 return_data=False)

In [ ]:
%%capture
thresh_model_2 = ep.ThresholdModel(g_bluetooth_contact)
thresh_config_2 = mc.Configuration()

threshold_2 = 0.35
fraction_infected_2 = .1

thresh_config_2.add_model_parameter("fraction_infected", fraction_infected_2)

for n in g_bluetooth_contact.nodes():
    thresh_config_2.add_node_configuration("threshold", n, threshold_2)
    
thresh_model_2.set_initial_status(thresh_config_2)

threshold_trends_2 = multi_runs(thresh_model_2,
                              execution_number=500,
                              iteration_number=40,
                              infection_sets=infect_sets, 
                              nprocesses=4)

In [ ]:
visualize_trends(threshold_trends_2,
                 network=g_bluetooth_contact,
                 states=[0, 1],
                 labels=['Not Activated', 'Activated'],
                 proportion=True,
                 return_data=False)

In [ ]:
%%capture
thresh_model_3 = ep.ThresholdModel(g_bluetooth_contact)
thresh_config_3 = mc.Configuration()

threshold_3 = 0.35
fraction_infected_2 = .15

thresh_config_3.add_model_parameter("fraction_infected", fraction_infected_2)

for n in g_bluetooth_contact.nodes():
    thresh_config_3.add_node_configuration("threshold", n, threshold_3)
    
thresh_model_3.set_initial_status(thresh_config_3)

threshold_trends_3 = multi_runs(thresh_model_3,
                              execution_number=500,
                              iteration_number=40,
                              infection_sets=infect_sets_2,
                              nprocesses=4)

In [ ]:
visualize_trends(threshold_trends_3,
                 network=g_bluetooth_contact,
                 states=[0, 1],
                 labels=['Not Activated', 'Activated'],
                 proportion=True,
                 return_data=False)

## Complex Contagions with Variable Thresholds


In [ ]:
from scipy.stats import truncnorm
def get_truncated_normal(mean, sd, low, upp):
    return truncnorm((low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [ ]:
model_cc_vt = ep.ThresholdModel(g_bluetooth_contact)
config_cc_vt = mc.Configuration()

fraction_infected = .1

random_thresholds = []
threshold_dist = get_truncated_normal(mean=.35, sd=.001, low=0, upp=1)

for n in g_bluetooth_contact.nodes():
    threshold = threshold_dist.rvs() 
    config_cc_vt.add_node_configuration("threshold", n, threshold)
    random_thresholds.append(threshold)
    
# with that done, we can add the other model information to the config.
config_cc_vt.add_model_parameter("fraction_infected", fraction_infected)

model_cc_vt.set_initial_status(config_cc_vt)

In [ ]:
sns.ecdfplot(random_thresholds)
sns.despine()
plt.show()

In [ ]:
%%capture
threshold_trends_vt = multi_runs(model_cc_vt,
                                 execution_number=500,
                                 iteration_number=40,
                                 infection_sets=infect_sets,
                                 nprocesses=4)

In [ ]:
visualize_trends(threshold_trends_vt,
                 network=g_bluetooth_contact,
                 states=[0, 1],
                 labels=['Not Activated', 'Activated'],
                 proportion=True,
                 return_data=False)

In [ ]:
%%capture
model_cc_vt_2 = ep.ThresholdModel(g_bluetooth_contact)
config_cc_vt_2 = mc.Configuration()

fraction_infected = .1

random_thresholds_2 = []
threshold_dist_2 = get_truncated_normal(mean=.35, sd=.1, low=0, upp=1)

for n in g_bluetooth_contact.nodes():
    threshold_2 = threshold_dist_2.rvs() 
    config_cc_vt_2.add_node_configuration("threshold", n, threshold_2)
    random_thresholds_2.append(threshold_2)
    
config_cc_vt_2.add_model_parameter("fraction_infected", fraction_infected)

model_cc_vt_2.set_initial_status(config_cc_vt_2)

threshold_trends_vt_2 = multi_runs(model_cc_vt_2,
                                 execution_number=500,
                                 iteration_number=40,
                                 infection_sets=infect_sets,
                                 nprocesses=4)


In [ ]:
visualize_trends(threshold_trends_vt_2,
                 network=g_bluetooth_contact,
                 states=[0, 1],
                 labels=['Not Activated', 'Activated'],
                 proportion=True,
                 return_data=False)

## Towards Greater Realism: Stochastic Thresholds and Network Diffusion Experiments


# CONCLUSION
## Key Points 
